In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['HK1']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [12]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [13]:
list(zip(longforms, counts))

[('hexokinase 1', 51),
 ('hexokinase i', 3),
 ('histidine aspartate kinase', 2),
 ('hong kong 1', 1),
 ('human keratin 1', 1),
 ('1 hexokinase', 1),
 ('hexokinase1', 1),
 ('human keratin 1 genes', 1),
 ('histidine aspartate kinase 1', 1),
 ('hexokinase 1 genes', 1),
 ('hexokinase isoforms 1', 1),
 ('hexokinase type 1', 1),
 ('kinase hks rv0600c', 1),
 ('hone1 3 × 103', 1),
 ('hr', 1),
 ('h2o2 induce apoptosis in npc', 1)]

In [14]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [15]:
names

{'CHEBI:CHEBI:17191': 'L-isoleucine',
 'HGNC:6412': 'KRT1',
 'HGNC:4922': 'HK1',
 'GO:GO:0000725': 'recombinational repair',
 'CHEBI:CHEBI:80552': 'NPC',
 'HGNC:6222': 'KCNA4',
 'HGNC:19884': 'HOOK1'}

In [16]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [17]:
result = [grounding_map, names, pos_labels]

In [18]:
result

[{'1 hexokinase': 'HGNC:4922',
  'h2o2 induce apoptosis in npc': 'ungrounded',
  'hexokinase 1': 'HGNC:4922',
  'hexokinase 1 genes': 'HGNC:4922',
  'hexokinase i': 'HGNC:4922',
  'hexokinase isoforms 1': 'HGNC:4922',
  'hexokinase type 1': 'HGNC:4922',
  'hexokinase1': 'HGNC:4922',
  'histidine aspartate kinase': 'UP:Q9SXL4',
  'histidine aspartate kinase 1': 'ungrounded',
  'hone1 3 × 103': 'ungrounded',
  'hong kong 1': 'ungrounded',
  'hr': 'ungrounded',
  'human keratin 1': 'HGNC:6412',
  'human keratin 1 genes': 'ungrounded',
  'kinase hks rv0600c': 'ungrounded'},
 {'HGNC:4922': 'HK1', 'UP:Q9SXL4': 'AHK1', 'HGNC:6412': 'KRT1'},
 ['HGNC:4922']]

In [51]:
grounding_map, names, pos_labels = [{'1 hexokinase': 'HGNC:4922',
  'h2o2 induce apoptosis in npc': 'ungrounded',
  'hexokinase 1': 'HGNC:4922',
  'hexokinase 1 genes': 'HGNC:4922',
  'hexokinase i': 'HGNC:4922',
  'hexokinase isoforms 1': 'HGNC:4922',
  'hexokinase type 1': 'HGNC:4922',
  'hexokinase1': 'HGNC:4922',
  'histidine aspartate kinase': 'UP:Q9SXL4',
  'histidine aspartate kinase 1': 'ungrounded',
  'hone1 3 × 103': 'ungrounded',
  'hong kong 1': 'ungrounded',
  'hr': 'ungrounded',
  'human keratin 1': 'HGNC:6412',
  'human keratin 1 genes': 'ungrounded',
  'kinase hks rv0600c': 'ungrounded'},
 {'HGNC:4922': 'HK1', 'UP:Q9SXL4': 'AHK1', 'HGNC:6412': 'KRT1'},
 ['HGNC:4922']]

In [52]:
excluded_longforms = ['hr', 'kinase hks rv0600c']

In [53]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [54]:
additional_entities = {'HGNC:19884': ['HOOK1', ['HK1', 'HOOK1', 'hook']]}

In [55]:
unambiguous_agent_texts = {}

In [56]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [57]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [58]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [59]:
intersection1

[('HGNC:19884', 'HGNC:19884', 45)]

In [60]:
intersection2

[('HGNC:4922', 'HGNC:19884', 0),
 ('HGNC:6412', 'HGNC:19884', 0),
 ('ungrounded', 'HGNC:19884', 0),
 ('UP:Q9SXL4', 'HGNC:19884', 0)]

In [61]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [62]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [63]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 04:41:18] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 04:41:19] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8532580348943984 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [64]:
classifier.stats

{'label_distribution': {'HGNC:4922': 47,
  'HGNC:6412': 1,
  'ungrounded': 5,
  'UP:Q9SXL4': 2,
  'HGNC:19884': 9},
 'f1': {'mean': 0.853258, 'std': 0.111044},
 'precision': {'mean': 0.80303, 'std': 0.144857},
 'recall': {'mean': 0.92697, 'std': 0.068316},
 'HGNC:19884': {'f1': {'mean': 0.533333, 'std': 0.452155},
  'pr': {'mean': 0.5, 'std': 0.447214},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'HGNC:6412': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:4922': {'f1': {'mean': 0.905455, 'std': 0.057668},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.832401, 'std': 0.098103}},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'UP:Q9SXL4': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}}}

In [65]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [66]:
disamb.dump(model_name, results_path)

In [67]:
print(disamb.info())

Disambiguation model for HK1

Produces the disambiguations:
	AHK1	UP:Q9SXL4
	HK1*	HGNC:4922
	HOOK1*	HGNC:19884
	KRT1	HGNC:6412

Class level metrics:
--------------------
Grounding	Count	F1     
  HK1*	47	0.90546
HOOK1*	 9	0.53333
Ungrounded	 5	    0.0
 AHK1	 2	    0.2
 KRT1	 1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.85326
	Precision:	0.80303
	Recall:		0.92697

* Positive labels
See Docstring for explanation



In [68]:
model_to_s3(disamb)

In [69]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [70]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:19884']

In [71]:
len(texts)

0